In [28]:
## scratch pad for citibike data set! exciting! get to break out my pandas knowledge
import pandas as pd
print(pd.__version__) #fortunately, latest version of pandas allows direct import from zip
import zipfile
import requests
import io
import time

0.19.0


In [2]:
## information on filename
citidir = "https://s3.amazonaws.com/tripdata/"
citi_extension = "-citibike-tripdata"
year = 2015
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [4]:
## store all data first in dictionary format using months as keys
d = {}

## download monthly data from the web for each month
for mth in months:
    #create filenames for given month
    filename = str(year) + mth + citi_extension
    citibike_zip_filename = citidir + filename + ".zip"
    citibike_csv_filename = filename + ".csv"
    print(citibike_zip_filename)
    
    #find file and use read_csv to save in dictionary of dataframes
    requested_file = requests.get(citibike_zip_filename)
    z = zipfile.ZipFile(io.BytesIO(requested_file.content))
    d[mth] = pd.read_csv(z.open(citibike_csv_filename), header=0, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201503-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201504-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201508-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201510-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip


In [64]:
## Concatenate into single yearly dataframe
df = pd.DataFrame()
tst=0

## append results of each month
for mth in months:
    tst += len(d[mth].index)
    print(tst)
    df = df.append(d[mth], ignore_index=True)

285552
482482
824308
1476698
2438684
3379903
4465579
5644623
6934322
8146599
9133844
9937969


In [74]:
df["tripduration"].median()

629.0

In [ ]:
df[df["ischild"] == df].value_counts(normalize = True)

In [62]:
print(d['12'].shape)

(804125, 15)


In [47]:
df = df.append(d['01'], ignore_index=True)
print(df.shape)

(285552, 15)


In [33]:
df.shape

(0, 0)

In [9]:
vertical_stack = pd.concat([d['01'], d['02']], axis=0)

In [14]:
vertical_stack.shape

(482482, 15)

In [15]:
ho_stack = pd.concat([d['01'], d['02']], axis=1)

In [27]:
ho_stack.iloc[[250000]]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
250000,427,1/25/2015 18:18,1/25/2015 18:25,527,E 33 St & 2 Ave,40.744023,-73.976056,440,E 45 St & 3 Ave,40.752554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for mth in months:
    citi_file = citidir + str(year) + mth + citi_extension + ".zip"
    print(citi_file)
    df2 = pd.read_csv(citi_file, compression='zip', header=1, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip


BadZipFile: File is not a zip file